In [1]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint="https://ai-asksulaimanov7666ai665127629473.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15",
    api_key="6RHqiKKJ4IlJNFHJu5KvBmb50LyaLREQuezH29sDYxNkhZKWdhevJQQJ99BEACfhMk5XJ3w3AAAAACOGhZNL"
)
import numpy as np

In [3]:
hello_embed=embeddings.embed_query("Hello, world!")
bye_embed=embeddings.embed_query("Bye, world!")
hi_embed=embeddings.embed_query("Hi, world!")

In [5]:
hello_embed = np.array(hello_embed)

In [6]:
hello_embed

array([-0.01914393, -0.02529205, -0.00172117, ..., -0.02266867,
        0.00564363, -0.01062804], shape=(1536,))

In [7]:
print(f"Hi and hello:{np.dot(hello_embed, hi_embed)}")
print(f"Bye and hello:{np.dot(bye_embed, hi_embed)}")
print(f"Hi and bye:{np.dot(bye_embed, hi_embed)}")

Hi and hello:0.8020760969287266
Bye and hello:0.6295159724598807
Hi and bye:0.6295159724598807


In [8]:
np.linalg.norm(bye_embed)

np.float64(1.0000000197725742)

In [13]:
family_text = "In the personal financial services industry, a family office is a private wealth management advisory company that serves ultra-high-net-worth individuals (UHNWI). In general, these are people with more than $30 million to invest."
family_embed = np.array(embeddings.embed_query(family_text))

In [15]:
print(f"family and hello:{np.dot(family_embed,hello_embed)}")
print(f"family and hi:{np.dot(family_embed,hi_embed)}")
print(f"family and bye:{np.dot(family_embed,bye_embed)}")

family and hello:0.0693441586717869
family and hi:0.12628418751517584
family and bye:0.12091690854937599


In [19]:
store_docs = [
    "Series A, B, and C are funding rounds that generally follow the stages of seed funding and angel investing, providing outside investors the opportunity to buy equity or partial ownership of a new company.",
    'The first round after the seed stage is Series A funding. The term gets its name from the preferred stock sold to investors at this stage.',
    'return on Investment (ROI) is a financial ratio that measures the profit generated by an investment relative to its cost. Key factors influencing ROI include the initial investment amount, ongoing maintenance costs, and the cash flow generated by the investment.',
    'market risk is the potential that entities may experience for financial losses stemming from changes in market prices.'
]

store_embedings = embeddings.embed_documents(store_docs)

print(store_embedings)

[[0.021727709099650383, -0.012079009786248207, 0.05445832014083862, 0.016347168013453484, 0.005773501005023718, 0.02361392043530941, 0.04967024177312851, 0.04422924667596817, 0.015585429035127163, 0.02373483218252659, 0.06654942035675049, -0.046236369758844376, -0.020361414179205894, -0.04650237411260605, -0.014847871847450733, 0.01761673390865326, -0.009044143371284008, -0.006057641468942165, -0.040747009217739105, 0.009177145548164845, 0.026866426691412926, 0.014908327721059322, 0.011547002010047436, 0.04587363824248314, -0.02865590900182724, -0.034677278250455856, -0.011262861080467701, 0.02117151767015457, -0.023299552500247955, -0.008312632329761982, 0.04768730327486992, -0.016951723024249077, -0.04933169111609459, 0.025729862973093987, -0.04038427770137787, 0.012212011963129044, -0.020965969190001488, 0.014231226406991482, 0.05876275151968002, 0.027204977348446846, 0.02355346456170082, 0.022295990958809853, 0.03153359144926071, -0.031461045145988464, 0.01382012851536274, -0.01631

In [20]:
store = np.array(store_embedings)


In [21]:
store

array([[ 0.02172771, -0.01207901,  0.05445832, ..., -0.01735073,
        -0.01308257,  0.02268291],
       [ 0.04697133, -0.02252447,  0.01316726, ..., -0.00208308,
         0.00040171,  0.0336072 ],
       [-0.0119533 ,  0.04418453,  0.01893318, ...,  0.00375675,
         0.03188971, -0.00052563],
       [ 0.01564354,  0.02981985,  0.05484539, ...,  0.01869166,
        -0.02428159,  0.00224734]], shape=(4, 1536))

In [33]:
def get_query(query):
    query_embed = np.array(embeddings.embed_query(query))
    query_embed = query_embed / np.linalg.norm(query_embed)

    score = np.dot(store_embedings, query_embed)
    index = np.argmax(score)
    results = {score: doc for score, doc in zip( score, store_docs)}

    return results, store_docs[index]

In [34]:
query = "Do investments have a return?"

results = get_query(query)

In [35]:
results

({np.float64(0.32914593995838826): 'Series A, B, and C are funding rounds that generally follow the stages of seed funding and angel investing, providing outside investors the opportunity to buy equity or partial ownership of a new company.',
  np.float64(0.2534930883791874): 'The first round after the seed stage is Series A funding. The term gets its name from the preferred stock sold to investors at this stage.',
  np.float64(0.4704615117523961): 'return on Investment (ROI) is a financial ratio that measures the profit generated by an investment relative to its cost. Key factors influencing ROI include the initial investment amount, ongoing maintenance costs, and the cash flow generated by the investment.',
  np.float64(0.27420467741238325): 'market risk is the potential that entities may experience for financial losses stemming from changes in market prices.'},
 'return on Investment (ROI) is a financial ratio that measures the profit generated by an investment relative to its cost.

In [37]:
import chromadb

chroma_client = chromadb.Client()

In [38]:
collection = chroma_client.get_or_create_collection(name="mine")

In [39]:
collection.add(documents=store_docs, ids= [f"id{x}" for x in range(len(store_docs))])

C:\Users\kampa\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [04:33<00:00, 304kiB/s]


In [42]:
collection.query(query_texts="What is an investor?", n_results=2)

{'ids': [['id0', 'id2']],
 'embeddings': None,
 'documents': [['Series A, B, and C are funding rounds that generally follow the stages of seed funding and angel investing, providing outside investors the opportunity to buy equity or partial ownership of a new company.',
   'return on Investment (ROI) is a financial ratio that measures the profit generated by an investment relative to its cost. Key factors influencing ROI include the initial investment amount, ongoing maintenance costs, and the cash flow generated by the investment.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.0745892524719238, 1.1267428398132324]]}